In [1]:
import pandas as pd
import numpy as np
import yaml
from tqdm import tqdm
from pathlib import Path
from glob import glob

import shapefile
import shapely
from shapely.geometry import Point

pd.set_option('max_columns', None)

In [2]:
df = pd.read_csv('../Data/GHS/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.csv')
df.head()

,ID_HDC_G0,QA2_1V,AREA,BBX_LATMN,BBX_LONMN,BBX_LATMX,BBX_LONMX,GCPNT_LAT,GCPNT_LON,CTR_MN_NM,CTR_MN_ISO,XBRDR,XCTR_NBR,XC_NM_LST,XC_ISO_LST,GRGN_L1,GRGN_L2,UC_NM_MN,UC_NM_LST,UC_NM_SRC,H75_NBR,H90_NBR,H00_NBR,H75_AREA,H90_AREA,H00_AREA,E_BM_NM_LST,E_SL_LST,EL_AV_ALS,E_KG_NM_LST,E_RB_NM_LST,E_WR_P_90,E_WR_P_00,E_WR_P_14,E_WR_T_90,E_WR_T_00,E_WR_T_14,B75,B90,B00,B15,P75,P90,P00,P15,BUCAP75,BUCAP90,BUCAP00,BUCAP15,NTL_AV,GDP90_SM,GDP00_SM,GDP15_SM,INCM_CMI,DEV_CMI,TT2CC,E_GR_AV90,E_GR_AV00,E_GR_AV14,E_GR_AH90,E_GR_AM90,E_GR_AL90,E_GR_AT90,E_GR_AH00,E_GR_AM00,E_GR_AL00,E_GR_AT00,E_GR_AH14,E_GR_AM14,E_GR_AL14,E_GR_AT14,E_EC2E_E75,E_EC2E_E90,E_EC2E_E00,E_EC2E_E15,E_EC2E_R75,E_EC2E_R90,E_EC2E_R00,E_EC2E_R15,E_EC2E_I75,E_EC2E_I90,E_EC2E_I00,E_EC2E_I15,E_EC2E_T75,E_EC2E_T90,E_EC2E_T00,E_EC2E_T15,E_EC2E_A75,E_EC2E_A90,E_EC2E_A00,E_EC2E_A15,E_EC2O_E75,E_EC2O_E90,E_EC2O_E00,E_EC2O_E15,E_EC2O_R75,E_EC2O_R90,E_EC2O_R00,E_EC2O_R15,E_EC2O_I75,E_EC2O_I90,E_EC2O_I00,E_EC2O_I15,E_EC2O_T75,E_EC2O_T90,E_EC2O_T00,E_EC2O_T15,E_EC2O_A75,E_EC2O_A90,E_EC2O_A00,E_EC2O_A15,E_EPM2_E75,E_EPM2_E90,E_EPM2_E00,E_EPM2_E15,E_EPM2_R75,E_EPM2_R90,E_EPM2_R00,E_EPM2_R15,E_EPM2_I75,E_EPM2_I90,E_EPM2_I00,E_EPM2_I15,E_EPM2_T75,E_EPM2_T90,E_EPM2_T00,E_EPM2_T15,E_EPM2_A75,E_EPM2_A90,E_EPM2_A00,E_EPM2_A15,E_CPM2_T00,E_CPM2_T05,E_CPM2_T10,E_CPM2_T14,EX_FD_AREA,EX_FD_B75,EX_FD_B90,EX_FD_B00,EX_FD_B15,EX_FD_P75,EX_FD_P90,EX_FD_P00,EX_FD_P15,EX_SS_AREA,EX_SS_B75,EX_SS_B90,EX_SS_B00,EX_SS_B15,EX_SS_P75,EX_SS_P90,EX_SS_P00,EX_SS_P15,EX_EQ19PGA,EX_EQ19MMI,EX_EQ19_Q,EX_HW_IDX,SDG_LUE9015,SDG_A2G14,SDG_OS15MX
0,1,1,185,21.247683,-158.043016,21.422193,-157.730529,21.340678,-157.893497,United States,USA,0,1,United States,USA,Northern America,Northern America,Honolulu,Honolulu; Waipahu; Pearl City; Aiea,GISCO OSM,1,1,1,115,152,174,Tropical and Subtropical Dry Broadleaf Forests,Vertisols,52.296435,NAN,NAN,955.500031,620.975021,741.625015,23.456032,23.153346,23.526622,78.845200,78.845200,80.214867,80.647377,277211.196645,378482.072123,458967.881664,512853.666675,284.422854,208.319509,174.772288,157.252219,24.768574,1.504144e+10,1.584126e+10,2.192668e+10,HIC,MDR,15412.057919,0.231515,0.349464,0.369290,3.310453,129.412130,51.089083,179.678081,56.331924,113.192717,14.287026,183.811667,70.317059,98.455611,15.038997,183.811667,2.867485e+06,1.265081e+06,1.012234e+06,495299.593436,456114.038821,403581.138607,462973.483368,396627.287294,598840.500466,601347.375094,541277.833862,456837.498271,208651.075028,276910.085019,341852.064389,345201.327942,0.013626,0.029494,0.042942,0.056613,NaN,156371.556763,42922.159473,11381.377166,NaN,13811.961727,11022.504909,8225.571851,NaN,29257.399703,67747.537985,51154.996318,NaN,NaN,1029.574596,10692.735959,2.418542,3.153900,2.293634,3.078804,300.287590,134.342385,98.071016,41.850553,88.248068,112.991622,68.033587,40.144556,118.767334,208.712524,224.997453,174.221274,29.250431,43.893797,29.364700,23.536766,0.042916,0.043009,0.041517,0.047534,7.237346,6.158930,5.563796,5.765125,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,161,68.280144,68.280144,69.576241,69.95092,240002.900655,327680.831522,397443.031445,444041.529529,NaN,NaN,missing,NaN,0.074385,0.226415,56.41
1,2,2,42,-17.641184,-149.628088,-17.517631,-149.508018,-17.534103,-149.568053,French Polynesia,PYF,0,1,French Polynesia,PYF,Oceania,Polynesia,Papeete,Papeete,GISCO OSM,0,1,1,0,29,29,Tropical and Subtropical Moist Broadleaf Forests,NAN,33.690318,NAN,NAN,1623.150055,1376.275009,1536.075012,21.907134,22.041168,22.021633,14.051702,14.051702,14.051702,14.493433,47644.964024,71862.231544,83726.092072,91521.124603,294.925221,195.536671,167.829421,158.361614,9.028501,0.000000e+00,0.000000e+00,0.000000e+00,HIC,LDC,NaN,NaN,0.445268,0.436477,0.000000,0.000000,0.000000,0.000000,15.656925,19.369919,6.614911,41.478138,16.868909,18.625840,6.147005,41.641755,3.240516e+03,2.106038e+04,1.506982e+04,4916.763772,4983.827260,16692.748949,3205.124311,1741.406966,1046.319099,4499.484418,8941.935806,3335.573413,4306.003194,9606.568066,1

In [3]:
iso_codes = pd.read_csv('../Data/iso_codes.csv')
iso_codes.loc[iso_codes['numeric'] == 516, 'iso2'] = 'NA'
iso_map = {r['iso3']: r for _, r in iso_codes.iterrows()}

In [4]:
shps = glob('../Shapefiles/preprocessed/*.shp')
shapes_by_country = {}
for i, shp in tqdm(enumerate(shps), total=len(shps)):
    codes = Path(shp).name.replace('.shp', '').split('_')
    shp = shapefile.Reader(shp)
    shapes = [shapely.geometry.shape(s) for s in shp.shapes()]
    records = shp.records()
    
    for code in codes:
        shapes_by_country[code] = []
        for shape, record in zip(shapes, records):
            shapes_by_country[code].append((shape, *record))
    
assert 'GT' in shapes_by_country and 'PM' in shapes_by_country

100%|██████████| 276/276 [02:13<00:00,  2.07it/s]


In [5]:
locs = df[['GCPNT_LON', 'GCPNT_LAT', 'CTR_MN_ISO']].drop_duplicates()
locs['adm1'] = np.nan
locs['adm2'] = np.nan
locs['nearest_loc'] = False

In [6]:
pbar = tqdm(total=locs.shape[0], desc='Finding codes')
for idx, row in locs.iterrows():
    if isinstance(row['adm1'], str):
        pbar.update(1)
        continue
    
    coordinates = (row['GCPNT_LON'], row['GCPNT_LAT'])
    point = Point(coordinates)
    nearest_record = None
    nearest_distance = np.inf
    for shape, adm0, adm1, adm2 in shapes_by_country.get(row['CTR_MN_ISO'], []):
        if point.within(shape):
            locs.loc[idx, 'adm2'] = adm2
            locs.loc[idx, 'adm1'] = adm1
            locs.loc[idx, 'adm0'] = adm0
            break
            
        distance = point.distance(shape)
        if distance < nearest_distance:
            nearest_distance = distance
            nearest_record = (adm0, adm1, adm2)
    else:
        if nearest_record:
            locs.loc[idx, 'nearest_loc'] = True
            locs.loc[idx, 'adm2'] = nearest_record[2]
            locs.loc[idx, 'adm1'] = nearest_record[1]
            locs.loc[idx, 'adm0'] = nearest_record[0]
        
    pbar.update(1)
pbar.close()
print('Total nearest:', locs['nearest_loc'].sum())
print('Missing:', locs.loc[locs['adm1'].isnull(), 'CTR_MN_ISO'].unique())

Finding codes: 100%|██████████| 13135/13135 [03:40<00:00, 59.58it/s] 

Total nearest: 103
Missing: []


In [7]:
df = df.merge(locs, on=['GCPNT_LON', 'GCPNT_LAT', 'CTR_MN_ISO'], how='left')

In [8]:
# adm = pd.read_csv('../Shapefiles/ADM/POLIO_ADMINISTRATIVE_BOUNDARIES.csv')
# adm.columns = [x.lower() for x in adm.columns]
# adm = adm[['adm0_name', 'adm1_name', 'adm2_name', 'adm1_code', 'adm2_code']].copy()
# adm.rename(columns={'adm1_code': 'adm1', 'adm2_code': 'adm2'}, inplace=True)

# df = df.merge(adm, on=['adm1', 'adm2'], how='left')
# locs_to_check = df.loc[
#     np.logical_or(df['adm1'].isnull(), df['nearest_loc']), 
#     ['CTR_MN_ISO', 'UC_NM_MN', 'adm1', 'adm2', 'adm0_name', 'adm1_name', 'adm2_name']
# ].drop_duplicates(['CTR_MN_ISO', 'UC_NM_MN'])

# print(locs_to_check.shape)

# locs_to_check.to_excel('../Output/GHS_locs_to_check.xlsx', index=False)

(281, 7)


/home/luisdasilva/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (32,33,34) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [8]:
print(df.shape)
df.to_csv('../Output/GHS.csv', index=False)

(13135, 164)
